In [1]:
import os
%pwd
os.chdir("../")
%pwd

'c:\\Users\\91629\\Desktop\\projects\\SmartPoultryMgt'

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Dict[str, str]  # Fix: Add this to accept train/test/valid paths
    all_params: Dict[str, Any]
    params_image_size: list
    params_batch_size: int


In [25]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [26]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data={  # <-- Now correctly mapped
                "train": "artifacts/data_ingestion/classification_dataset/train",
                "test": "artifacts/data_ingestion/classification_dataset/test",
                "valid": "artifacts/data_ingestion/classification_dataset/valid"
            },
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [27]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [28]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [31]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data["valid"],

            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    


In [32]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.save_score()
  

except Exception as e:
   raise e

[2025-03-18 05:19:22,613: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-18 05:19:22,618: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-18 05:19:22,620: INFO: common: created directory at: artifacts]
Found 928 images belonging to 2 classes.
29/29 [==============================] - 22s 261ms/step - loss: 2.1693 - accuracy: 0.8599
[2025-03-18 05:19:45,472: INFO: common: json file saved at: scores.json]
